# IGA-05. Initial Steps in Image Recognition

**Students**
- Dodiya Indraraj
- Magomedova Zamira
- Makarkina Irina
- Versotskaya Nataliya

**Problem setup**

In this assignment you are asked to apply ANNs in order to classify a given silhouette (image) of a vehicle as one of four types
-	a double decker bus (class 0),
-	Opel Manta 400 (class 1),
-	Saab 9000 (class 2), 
-	Cheverolet van (class 3),

using a set of features extracted from the silhouette. The vehicle may be viewed from one of many different angles. The dataset is given in two files: **veh_train.csv** and **veh_test.csv**. Variable class is the outcome (nominal) variable.

One of the principal questions of this assignment is whether PCA can help (or, at least, not make worse) predicting classes.

Data source and detailed description: https://archive.ics.uci.edu/ml/datasets/Statlog+%28Vehicle+Silhouettes%29 



In [1]:
# load traditional libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.decomposition import PCA as skPCA
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report

In [2]:
# load KERAS
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import np_utils

Using TensorFlow backend.


In [3]:
# set constants
c_nclass = 4

list_feats = ["V1", "V2", "V3","V4","V5","V6","V7","V8","V9","V10","V11","V12","V13","V14","V15","V16",
                             "V17","V18"]

In [4]:
# define a function for your ANN
def myANN(input_dim):
    
    model = Sequential()
    model.add(Dense(128, input_dim=input_dim, activation='relu'))
    model.add(Dense(c_nclass, activation="softmax"))
    model.compile(loss='categorical_crossentropy', optimizer="adam", metrics=['accuracy'])
    
    return model

### Reading and preparing the data

In [5]:
# read the data
veh_train = pd.read_csv('veh_train.csv', header=0)
veh_test = pd.read_csv('veh_test.csv', header=0)

In [6]:
# construct the training and the test samples
X_train = veh_train.drop('class', axis=1)
y_train = veh_train['class']

X_test = veh_test.drop('class', axis=1)
y_test = veh_test['class']

In [7]:
# since PCA will be applied, let's scale the features

# scale the training data
X_train_sc = StandardScaler().fit_transform(X_train)
X_train_sc = pd.DataFrame(X_train_sc)
X_train_sc.columns = list_feats

# scale the test data
X_test_sc = StandardScaler().fit_transform(X_test)
X_test_sc = pd.DataFrame(X_test_sc)
X_test_sc.columns = list_feats

# preview the scaled data
X_train_sc.head(6)

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18
0,0.142657,0.493358,0.048570,0.266144,1.350583,0.331154,-0.211507,0.142691,-0.233227,0.751569,-0.406215,-0.350463,0.268827,-0.318038,-0.084475,0.389751,-0.334073,0.170051
1,1.241760,0.817326,1.519254,1.204229,0.563227,0.331154,1.140490,-1.135304,0.923806,0.682146,1.099655,1.094856,1.360598,0.096104,1.551923,-0.418356,-0.172632,0.036608
2,-0.101589,-0.640530,-0.015373,-0.308812,0.169549,0.103675,-0.752306,0.653888,-0.618905,-0.359197,-0.918852,-0.745667,-1.459809,-1.284369,-0.084475,-0.302912,1.603217,1.504479
3,-1.078570,-0.154578,-0.782686,1.083186,5.418592,9.885314,-0.602084,0.526089,-0.618905,-0.289774,1.676371,-0.655335,0.390135,7.550654,0.529174,-0.187469,-1.464159,-1.698148
4,1.608128,1.951214,1.519254,0.084579,-1.536390,-0.578765,2.582619,-1.902101,2.852195,1.445798,2.925923,2.912796,2.694984,1.752670,-0.289024,-0.418356,-1.302718,-1.698148
5,-0.467957,-0.316562,-1.038457,-0.369334,0.432001,0.103675,-0.962616,0.909487,-1.004582,-0.150928,-0.854772,-0.903749,-0.337712,-0.732180,-0.698124,-1.111019,0.634572,0.837265


## Task 1 (1 point). Descriptive analysis of the ORIGINAL training sample

Read the description of the features. Analyze the structure and features of the dataset (no graphs!). Do the following on the **training sample**:

-	Check whether the classes are balanced.
-	Calculate summary statistics and correlations of the ORIGINAL features.
-	Do you expect that PCA may be helpful? Give brief comments.


In [8]:
# check the balance of classes
y_test.value_counts(normalize=True)

1    0.279762
2    0.255952
3    0.232143
0    0.232143
Name: class, dtype: float64

In [9]:
# summary statistics on features
X_test.describe()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18
count,168.000000,168.000000,168.000000,168.00000,168.000000,168.000000,168.000000,168.000000,168.000000,168.000000,168.000000,168.000000,168.000000,168.000000,168.000000,168.00000,168.000000,168.000000
mean,93.059524,44.488095,81.476190,167.87500,61.636905,8.660714,168.029762,41.136905,20.494048,147.291667,188.410714,435.214286,172.958333,73.101190,6.232143,12.50000,188.380952,195.255952
std,8.389973,6.141045,16.284136,35.12905,8.892859,5.353191,33.073213,7.754408,2.587388,14.937536,32.128733,174.852963,30.687795,8.371351,5.033984,9.94837,6.007789,7.198633
min,77.000000,33.000000,42.000000,104.00000,47.000000,3.000000,114.000000,26.000000,17.000000,118.000000,131.000000,191.000000,118.000000,61.000000,0.000000,0.00000,176.000000,182.000000
25%,86.000000,40.000000,70.000000,141.00000,56.750000,6.750000,143.000000,33.000000,19.000000,136.000000,167.000000,307.750000,149.750000,68.000000,2.000000,5.00000,184.000000,190.750000
50%,92.000000,43.000000,77.500000,167.00000,61.000000,8.000000,155.500000,43.000000,19.000000,144.000000,177.000000,359.000000,172.000000,72.000000,5.000000,9.50000,189.000000,196.500000
75%,99.000000,49.250000,98.000000,192.25000,65.000000,10.000000,200.250000,46.000000,23.000000,160.000000,219.000000,599.000000,189.000000,76.000000,10.000000,20.00000,192.000000,201.000000
max,119.000000,58.000000,112.000000,333.00000,138.000000,55.000000,252.000000,59.000000,28.000000,182.000000,320.000000,928.000000,246.000000,135.000000,22.000000,40.00000,203.000000,210.000000


In [10]:
# correlation matrix
X_test.corr()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18
V1,1.000000,0.699542,0.788050,0.654878,0.080548,0.135643,0.815838,-0.799949,0.817338,0.679767,0.732909,0.820336,0.584789,-0.266598,0.236441,0.181148,0.338834,0.419925
V2,0.699542,1.000000,0.818846,0.599700,0.151837,0.234394,0.859786,-0.824165,0.859801,0.972376,0.770759,0.854380,0.933732,-0.015410,0.136358,0.075177,-0.023086,0.153200
V3,0.788050,0.818846,1.000000,0.740456,0.157877,0.238783,0.915696,-0.920770,0.899692,0.793874,0.841594,0.902781,0.712613,-0.273401,0.098208,0.307754,0.221113,0.430547
V4,0.654878,0.599700,0.740456,1.000000,0.707381,0.516383,0.707801,-0.759417,0.672070,0.552712,0.838437,0.703112,0.515483,-0.086557,0.065924,0.159254,0.436090,0.520999
V5,0.080548,0.151837,0.157877,0.707381,1.000000,0.696259,0.089659,-0.160701,0.052084,0.129094,0.403399,0.084352,0.136533,0.293844,0.008716,-0.071644,0.278881,0.288531
V6,0.135643,0.234394,0.238783,0.516383,0.696259,1.000000,0.147452,-0.158561,0.131928,0.275023,0.426961,0.132797,0.177975,0.379453,0.070047,0.021420,0.050590,0.212975
V7,0.815838,0.859786,0.915696,0.707801,0.089659,0.147452,1.000000,-0.974535,0.991869,0.823876,0.903663,0.997124,0.788788,-0.126122,0.076710,0.300553,0.085229,0.231232
V8,-0.799949,-0.824165,-0.920770,-0.759417,-0.160701,-0.158561,-0.974535,1.000000,-0.950079,-0.784625,-0.901221,-0.961996,-0.750600,0.203276,-0.067854,-0.255725,-0.209352,-0.328024
V9,0.817338,0.859801,0.899692,0.672070,0.052084,0.131928,0.991869,-0.950079,1.000000,0.828083,0.881959,0.992103,0.789099,-0.114010,0.097801,0.304167,0.051381,0.202784
V10,0.679767,0.972376,0.793874,0.552712,0.129094,0.275023,0.823876,-0.784625,0.828083,1.000000,0.726810,0.814325,0.876480,-0.018817,0.114243,0.089959,-0.026134,0.182122


**Give your comments here**

Based on descriptive statisctic we see that there are no missing values in the dataset

We see high correlation, for example, between V2 (compactness) & V10 (MAX.LENGTH RECTANGULARITY); V2(compactness)&V13(SCALED RADIUS OF GYRATION) etc.

V8 (ELONGATEDNESS) has negative correlation with all the features, the strongest is with V7 (SCATTER RATIO), and also high with V3 (DISTANCE CIRCULARITY), V11 (SCALED VARIANCE along major x) & V12 (SCALED VARIANCE along minor x)

For PCA variance is important, we see in initial data that features V15, V16, V6 have one-two digit values, while V12 3digit values that are quite high, however stdev is somewhat similar.
 
Classes are somewhat balanced.

We expect PCA to be somewhat helpful as there are some some features that are highly correlated with each other.



## Task 2 (1 point). ANN on the SCALED full dataset

Estimate and validate a simple 1-layer ANN on the SCALED full data. Do the following:

- Transform the outcome variable on both sample appropriately.
- Run the model (use 100 epochs).
- Validate its performance.
- Give brief comments.



In [11]:
# prepare the outcome variable
Y_01_train = np_utils.to_categorical(y_train, c_nclass)
Y_01_test = np_utils.to_categorical(y_test, c_nclass) 

In [12]:
# setup the Model1 with the myANN function
model1 = myANN(18)

In [13]:
# train Model1
np.random.seed(12345) # do not change it!
model1.fit(X_train_sc, Y_01_train, epochs=100, validation_data = (X_test_sc, Y_01_test))

Train on 678 samples, validate on 168 samples
Epoch 1/100
678/678 [==============================] - 1s 2ms/step - loss: 1.2541 - accuracy: 0.4602 - val_loss: 1.1833 - val_accuracy: 0.4821
Epoch 2/100
678/678 [==============================] - 0s 117us/step - loss: 1.0451 - accuracy: 0.5590 - val_loss: 1.0675 - val_accuracy: 0.5595
Epoch 3/100
678/678 [==============================] - 0s 117us/step - loss: 0.9257 - accuracy: 0.6342 - val_loss: 0.9796 - val_accuracy: 0.6369
Epoch 4/100
678/678 [==============================] - 0s 146us/step - loss: 0.8420 - accuracy: 0.6770 - val_loss: 0.9275 - val_accuracy: 0.6190
Epoch 5/100
678/678 [==============================] - 0s 124us/step - loss: 0.7748 - accuracy: 0.6932 - val_loss: 0.8595 - val_accuracy: 0.6845
Epoch 6/100
678/678 [==============================] - 0s 116us/step - loss: 0.7157 - accuracy: 0.7448 - val_loss: 0.8180 - val_accuracy: 0.6905
Epoch 7/100
678/678 [==============================] - 0s 112us/step - loss: 0.6667 - 

Epoch 57/100
678/678 [==============================] - 0s 96us/step - loss: 0.2691 - accuracy: 0.9041 - val_loss: 0.4896 - val_accuracy: 0.7917
Epoch 58/100
678/678 [==============================] - 0s 112us/step - loss: 0.2643 - accuracy: 0.9071 - val_loss: 0.4866 - val_accuracy: 0.7857
Epoch 59/100
678/678 [==============================] - 0s 160us/step - loss: 0.2619 - accuracy: 0.9056 - val_loss: 0.4976 - val_accuracy: 0.7738
Epoch 60/100
678/678 [==============================] - 0s 165us/step - loss: 0.2566 - accuracy: 0.9204 - val_loss: 0.5067 - val_accuracy: 0.7857
Epoch 61/100
678/678 [==============================] - 0s 165us/step - loss: 0.2592 - accuracy: 0.9012 - val_loss: 0.4829 - val_accuracy: 0.7857
Epoch 62/100
678/678 [==============================] - 0s 159us/step - loss: 0.2589 - accuracy: 0.9086 - val_loss: 0.4888 - val_accuracy: 0.7857
Epoch 63/100
678/678 [==============================] - 0s 158us/step - loss: 0.2500 - accuracy: 0.9145 - val_loss: 0.4896 - 

In [14]:
# prediction of classes
Y_01_test_pred = model1.predict(X_test_sc)
Y_01_test_pred[0]
# classification report
print(classification_report(y_true=Y_01_test, y_pred=Y_01_test_pred.round(0)))

              precision    recall  f1-score   support

           0       0.97      0.97      0.97        39
           1       0.69      0.57      0.63        47
           2       0.62      0.65      0.64        43
           3       0.88      0.95      0.91        39

   micro avg       0.79      0.77      0.78       168
   macro avg       0.79      0.79      0.79       168
weighted avg       0.78      0.77      0.78       168
 samples avg       0.77      0.77      0.77       168



/Users/useruser/anaconda3/envs/tensorflow_env/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1268: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


**Give your comments here**

So the final accuracy is 77,98%, which is quite low, and classification report shows good results, predicting 0 and 3rd classes with high accuracy.

We expect that maybe PCA analysis will provide us with a better quality of classification of class 1 and 2, due to high correlation of features.

## Task 3 (2 points). PCA analysis

- Run a PCA analysis on the scaled features.
- Calculate variances and cumulative variances.
- For further analysis, leave only first 9 principal components. How much variance of the original dataset do they explain?
- Give brief comments.

In [15]:
# run a PCA
pca = skPCA(n_components=None)
pca.fit(X_train_sc)

PCA(copy=True, iterated_power='auto', n_components=None, random_state=None,
    svd_solver='auto', tol=0.0, whiten=False)

In [16]:
# % of variance explained by PCs
var_exp = pca.explained_variance_ratio_
var_exp.round(3)

array([0.524, 0.173, 0.101, 0.067, 0.049, 0.03 , 0.02 , 0.012, 0.009,
       0.005, 0.003, 0.002, 0.002, 0.001, 0.001, 0.001, 0.   , 0.   ])

In [17]:
# cumulative variance
cum_var_exp = np.cumsum(var_exp)
cum_var_exp

array([0.52367961, 0.69650739, 0.7979649 , 0.8647294 , 0.91364211,
       0.94337965, 0.96327801, 0.97549201, 0.98431741, 0.98941662,
       0.9926432 , 0.99506694, 0.99702208, 0.99810084, 0.99892997,
       0.99963912, 0.99998021, 1.        ])

In [18]:
# make first 9 components - for the training and test samples
pca = skPCA(n_components=9) #we leave only 9 components
pca.fit(X_train_sc)
PC_X_train = pca.transform(X_train_sc)
PC_X_test = pca.transform(X_test_sc)

**Give your comments here**

So, the first 9 variables explain 98% of the variance, so we expect that leaving only those 9 components will provide better classification results.

## Task 4 (1 point). ANN on the PCA-transformed data

- Re-run the ANN on the chosen 9 PCs.
- Estimate its performance on the test data.
- Give brief comments on the resulting performance.

In [19]:
# setup the Model2 with the myANN function
model2 = myANN(9)

In [20]:
# train Model1
np.random.seed(67890)
model2.fit(PC_X_train, Y_01_train, epochs = 100, validation_data = (PC_X_test, Y_01_test))

Train on 678 samples, validate on 168 samples
Epoch 1/100
678/678 [==============================] - 1s 2ms/step - loss: 1.3390 - accuracy: 0.3274 - val_loss: 1.2514 - val_accuracy: 0.4345
Epoch 2/100
678/678 [==============================] - 0s 123us/step - loss: 1.1296 - accuracy: 0.5560 - val_loss: 1.1475 - val_accuracy: 0.5595
Epoch 3/100
678/678 [==============================] - 0s 133us/step - loss: 1.0103 - accuracy: 0.6519 - val_loss: 1.0634 - val_accuracy: 0.6071
Epoch 4/100
678/678 [==============================] - 0s 133us/step - loss: 0.9190 - accuracy: 0.7021 - val_loss: 0.9844 - val_accuracy: 0.6429
Epoch 5/100
678/678 [==============================] - 0s 133us/step - loss: 0.8410 - accuracy: 0.7242 - val_loss: 0.9323 - val_accuracy: 0.6369
Epoch 6/100
678/678 [==============================] - 0s 127us/step - loss: 0.7784 - accuracy: 0.7463 - val_loss: 0.8839 - val_accuracy: 0.6429
Epoch 7/100
678/678 [==============================] - 0s 134us/step - loss: 0.7233 - 

Epoch 57/100
678/678 [==============================] - 0s 95us/step - loss: 0.3425 - accuracy: 0.8407 - val_loss: 0.6465 - val_accuracy: 0.7262
Epoch 58/100
678/678 [==============================] - 0s 96us/step - loss: 0.3391 - accuracy: 0.8555 - val_loss: 0.6488 - val_accuracy: 0.7321
Epoch 59/100
678/678 [==============================] - 0s 112us/step - loss: 0.3359 - accuracy: 0.8525 - val_loss: 0.6512 - val_accuracy: 0.7083
Epoch 60/100
678/678 [==============================] - 0s 91us/step - loss: 0.3331 - accuracy: 0.8451 - val_loss: 0.6455 - val_accuracy: 0.7143
Epoch 61/100
678/678 [==============================] - 0s 90us/step - loss: 0.3341 - accuracy: 0.8496 - val_loss: 0.6393 - val_accuracy: 0.7143
Epoch 62/100
678/678 [==============================] - 0s 94us/step - loss: 0.3302 - accuracy: 0.8525 - val_loss: 0.6474 - val_accuracy: 0.7262
Epoch 63/100
678/678 [==============================] - 0s 94us/step - loss: 0.3279 - accuracy: 0.8540 - val_loss: 0.6472 - val_a

In [21]:
# prediction of classes
Y_01_test_pred1 = model2.predict(PC_X_test)
# classification report
print(classification_report(y_true=Y_01_test, y_pred=Y_01_test_pred1.round(0)))

              precision    recall  f1-score   support

           0       0.95      0.97      0.96        39
           1       0.60      0.53      0.56        47
           2       0.54      0.49      0.51        43
           3       0.88      0.92      0.90        39

   micro avg       0.74      0.71      0.73       168
   macro avg       0.74      0.73      0.73       168
weighted avg       0.73      0.71      0.72       168
 samples avg       0.71      0.71      0.71       168



/Users/useruser/anaconda3/envs/tensorflow_env/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1268: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


**Give your comments here**

The new accuracy is 72.62%, which is lower than the accuracy without the PCA, and the classification report shows worse results, compared to the previous one. Recalls for all classes, except 0, became lower, and precision dropped for all classes, except for the 3rd one.

We don't think that PCA is very reasonable to conduct in this case, because all the metrics are lower than without it.

We tried using only 4 principal components, because they explain more than 80% of variance, which is considered a floor in PCA, and the results were even worse